In [6]:
import pandas as pd
from source import data_import
from source.constants import CHANNELS

print(f"loaded channel names: {CHANNELS}")

loaded channel names: ['F4-C4', 'F3-C3', 'FT9-FT10', 'FZ-CZ', 'F7-T7', 'FP2-F4', 'T8-P8-1', 'T8-P8-0', 'FP1-F3', 'CZ-PZ']


In [7]:
# load data
df_pp = data_import.load_pyarrow(file_name="preprocessed_df")

/Users/Samet/Documents/Bildung/BootcampSpiced/github_rice_regression/Capstone_project/ai2/source/../data/preprocessed_df.arrow was loaded.


## Feature Extraction

In [14]:
df_features = df_pp.groupby(['seizure_id', 'segment_id']).agg(
    {C:['mean','var', 'std'] for C in CHANNELS} | 
    {'is_seizure': 'first'}) # just taking first element for target column
df_features.head()

channel                   F4-C4                            F3-C3              \
                           mean         var        std      mean         var   
seizure_id segment_id                                                          
0          1           0.642823  638.991344  25.278278 -0.136380  816.655466   
           2           0.459926  620.426452  24.908361  0.607864  519.355782   
           3          -0.336786  628.921793  25.078313  0.020483  586.056809   
           4           0.571550  566.260437  23.796227  0.170771  732.239825   
           5          -0.156618  785.695596  28.030262  0.019398  822.613674   

channel                           FT9-FT10                            FZ-CZ  \
                             std      mean         var        std      mean   
seizure_id segment_id                                                         
0          1           28.577184  0.109270  776.888255  27.872715  0.072943   
           2           22.789379  0.984358  551.560383  23.485323  0.457697   
           3           24.208610 -0.237448  429.868368  20.733267 -0.281956   
           4           27.059930 -0.130328  572.179434  23.920272  0.457751   
           5           28.681243  0.157825  632.347132  25.146513  0.097109   

channel                ...   T8-P8-0                           FP1-F3  \
                       ...      mean         var        std      mean   
seizure_id segment_id  ...                                              
0          1           ...  0.365832  509.994709  22.583062 -0.383942   
           2           ...  0.455336  454.314548  21.314656  1.242494   
           3           ... -0.051306  402.037937  20.050884 -0.092616   
           4           ...  0.214385  393.301219  19.831823  0.136081   
           5           ... -0.188304  463.131613  21.520493 -0.279403   

channel                                          CZ-PZ               \
                              var        std      mean          var   
seizure_id segment_id                                                 
0          1           704.741392  26.546966  0.758870  1256.540374   
           2           654.580741  25.584776  0.040398  1146.577756   
           3           505.928076  22.492845  0.139786   928.583879   
           4           623.521637  24.970415  0.347514   948.839359   
           5           838.658633  28.959603 -0.032786  1530.441051   

channel                          is_seizure  
                             std      first  
seizure_id segment_id                        
0          1           35.447713      False  
           2           33.861154      False  
           3           30.472674      False  
           4           30.803236      False  
           5           39.120852      False  

[5 rows x 31 columns]

## Flatten Dataframe

In [12]:
# joining column names with agg functions, but leaving target 'is_seizure' column as 'is_seizure'.
df_features.columns = ['_'.join(col).strip() for col in df_features.columns.values if 'is_seizure' != col[0]] + ['is_seizure']
df_features.reset_index(inplace=True)
df_features.head(20)

,seizure_id,segment_id,F4-C4_var,F4-C4_std,F3-C3_var,F3-C3_std,FT9-FT10_var,FT9-FT10_std,FZ-CZ_var,FZ-CZ_std,...,FP2-F4_std,T8-P8-1_var,T8-P8-1_std,T8-P8-0_var,T8-P8-0_std,FP1-F3_var,FP1-F3_std,CZ-PZ_var,CZ-PZ_std,is_seizure
0,0,1,638.991344,25.278278,816.655466,28.577184,776.888255,27.872715,1612.539754,40.156441,...,25.380978,509.994709,22.583062,509.994709,22.583062,704.741392,26.546966,1256.540374,35.447713,False
1,0,2,620.426452,24.908361,519.355782,22.789379,551.560383,23.485323,1086.381750,32.960306,...,23.324690,454.314548,21.314656,454.314548,21.314656,654.580741,25.584776,1146.577756,33.861154,False
2,0,3,628.921793,25.078313,586.056809,24.208610,429.868368,20.733267,1077.790492,32.829720,...,18.870577,402.037937,20.050884,402.037937,20.050884,505.928076,22.492845,928.583879,30.472674,False
3,0,4,566.260437,23.796227,732.239825,27.059930,572.179434,23.920272,1160.493667,34.066019,...,22.235792,393.301219,19.831823,393.301219,19.831823,623.521637,24.970415,948.839359,30.803236,False
4,0,5,785.695596,28.030262,822.613674,28.681243,632.347132,25.146513,1772.292042,42.098599,...,29.556387,463.131613,21.520493,463.131613,21.520493,838.658633,28.959603,1530.441051,39.120852,False
5,0,6,1586.879066,39.835651,1129.883483,33.613739,669.451556,25.873762,2213.607867,47.048994,...,35.851282,486.567721,22.058280,486.567721,22.058280,1144.591089,33.831806,1433.627065,37.863268,False
6,0,7,1115.365450,33.397087,1189.425763,34.488052,724.126253,26.909594,2077.111833,45.575342,...,32.604831,558.245441,23.627218,558.245441,23.627218,1274.697172,35.702901,1456.325393,38.161832,False
7,0,8,1014.337394,31.848664,799.784463,28.280461,687.830348,26.226520,1432.826360,37.852693,...,35.244524,530.930428,23.041928,530.930428,23.041928,1236.486973,35.163717,1391.243264,37.299374,False
8,0,9,831.306319,28.832383,847.837468,29.117649,596.640558,24.426227,1632.812758,40.408078,...,30.023711,458.348241,21.409069,458.348241,21.409069,678.033401,26.039075,1037.524122,32.210621,False
9,0,10,1561.490081,39.515694,1617.036153,40.212388,647.497774,25.445978,2980.068901,54.590007,...,42.920719,504.959214,22.471298,504.959214,22.471298,1409.155721,37.538723,1436.193366,37.897142,False


In [13]:
from source.data_import import save_pyarrow

save_pyarrow(df_features, file_name="feature_extracted")

/Users/Samet/Documents/Bildung/BootcampSpiced/github_rice_regression/Capstone_project/ai2/source/../data/feature_extracted.arrow was successfully written.
